[Chapter 4] Optimal Clustering

In [ ]:
# a clustering problem consists of a set of objects and a set of features associated with those objects (unsupervised learning)
# the goal is to seperate the objects into groups (called clusters) using the features
#           where intragroup similarities are maximized and intergroup similarities are minimized

# 2 main classes of clustering: partitional (one-level/un-nested) and hierarchical(nested sequence of partitions)
# types of clustering algo: connectivity (e.g. hierarchical clustering), centroids (e.g. k-means), distribution,
#                      density (e.g. DBSCAN, OPTICS), subspace (on 2-dimensions both features and observations, e.g. bi/co-clustering) 

import numpy as np
import pandas as pd

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples

base clustering

In [7]:
def clusterKMeansBase(corr0, maxNumClusters=10, n_init=10):
    """
    use k-means algorithm on observation matrix
    """
    # derive observation matrix x as distance matrix
    x = ((1-corr0.fillna(0))/2.)**.5    # corr=1, x=0; corr=-1, x=1
    silh = pd.Series()
    
    for init in range(n_init):  # k-means uses multiple random initialization to avoid local optimas
        for i in range(2, maxNumClusters+1): # loop different number of maxNumClusters
            
            # k-means clustering
            kmeans_ = KMeans(n_clusters=i, n_jobs=1, n_init=1)  # only initialization once for inner loop
            kmeans_ = kmeans_.fit(x)

            #  calculate silhouette coef in measuring comparing intracluster distance and intercluster distance
            #       Si = (b_i-a_i)/max{a_i, b_i} 
            #       - a_i is the avg distance between i and all other elements in the same cluster
            #       - b_i is the avg distance between i and all the elements in the nearest cluster which i is not a member 
            #       - Si = 1 means i was clustered well and -1 means i was clustered poorly
            silh_ = silhouette_samples(x, kmeans_.labels_)
            
            # clustering quality q = mean(silh)/std(silh)
            # comparing current q_ vs historical optimal q
            stat = (silh_.mean()/silh_.std(), silh.mean()/silh.std())
            if np.isnan(stat[1]) or stat[0]>stat[1]:    
                silh, kmeans=silh_, kmeans_     # select clustering with the highest q

    # reorder correlation matrix based on kmeans clustering
    newIdx = np.argsort(kmeans.labels_)
    corr1 = corr0.iloc[newIdx] # reorder rows
    corr1 = corr1.iloc[:, newIdx] # reorder columns

    # extract/output clustering info
    clstrs = {i:corr0.columns[np.where(kmeans.labels_==i)[0]].tolist() \
              for i in np.unique(kmeans.labels_)}   # cluster members
    
    silh = pd.Series(silh, index=x.index)
    
    return corr1, clstrs, silh

higher-level clustering

In [9]:
def makeNewOutputs(corr0, clstrs, clstrs2):
    """
    combine two sets of clusters into one unified clustering result.
    update the correlation matrix and silhouette scores accordingly.
    """
    # take two dictionaries of clusters and merge these groups into a single new collection of clusters
    clstrsNew = {}
    for i in clstrs.keys():
        clstrsNew[len(clstrsNew.keys())] = list(clstrs[i])
    for i in clstrs2.keys():
        clstrsNew[len(clstrsNew.keys())] = list(clstrs2[i])
    
    # get new idx of clstrsNew (outer loop i to loop each keys in cltrsNew, inner loop to loop each item for each key) 
    newIdx = [j for i in clstrsNew for j in clstrsNew[i]]
    # extract the relevant subset of the original correlation matrix that corresponds to all items in the new combined clusters
    corrNew = corr0.loc[newIdx, newIdx]
    
    # convert the correlation matrix into distance matrix 
    x = ((1-corr0.fillna(0))/2.)**.5
    # assign cluster labels to each item in this new combined set
    kmeans_labels = np.zeros(len(x.columns))
    for i in clstrsNew.keys():
        idxs = [x.index.get_loc(k) for k in clstrsNew[i]]
        kmeans_labels[idxs] = i
    # calculate silhouette scores, which measure how well each item fits within its cluster.
    silhNew = pd.Series(silhouette_samples(x, kmeans_labels), index=x.index)
    # returns the updated correlation matrix, combined clusters, and silhouette scores
    return corrNew, clstrsNew, silhNew

#---------------------------------------------------------------------------------

def clusterKMeansTop(corr0, maxNumClusters=None, n_init=10):
    """
    perform K-means clustering on the correlation matrix.
    identify weak clusters based on silhouette scores.
    recursively refine those weak clusters by re-clustering them.
    decide whether the refinement improves clustering quality or not.
    """
    # intial clustering
    # run base K-means clustering on full dataset and get initial clusters and silh scores
    if maxNumClusters==None: maxNumClusters=corr0.shape[1]-1
    corr1, clstrs, silh=clusterKMeansBase(corr0, maxNumClusters=\
        min(maxNumClusters, corr0.shape[1]-1), n_init=n_init)
    
    # evaluate clusters
    # calculate qualtiy(t-stat like) score for each cluster
    clusterTstats = {i:np.mean(silh(clstrs[i])) / \
        np.std(silh[clstrs[i]]) for i in clstrs.keys()}
    # compute the average quality across all clusters
    tStatMean = sum(clusterTstats.values())/len(clusterTstats)
    
    # identify weak clusters
    # find the set of weak clusters with quality below average
    redoClusters = [i for i in clusterTstats.keys() if \
        clusterTstats[i]<tStatMean]
    # refine the weak clusters (redoClusters)
    if len(redoClusters)<=1:
        return corr1, clstrs, silh
    else:
        # if multiple weak clusters
        # extract items in those weak clusters
        keysRedo = [j for i in redoClusters for j in clstrs[i]]
        corrTmp = corr0.loc[keysRedo, keysRedo]
        tStatMean = np.mean([clusterTstats[i] for i in redoClusters])
        # run the clustering function recursively on just those items, aiming to split or reorganize them into better clusters.
        corr2, clstrs2, silh2 = clusterKMeansTop(corrTmp, \
            maxNumClusters=min(maxNumClusters, corrTmp.shape[1]-1), n_init=n_init)
        # make new outputs
        # merge the strong clusters (clstrs no in redoClusters) with the refined weak clusters (clstrs2)
        corrNew, clstrsNew, silhNew = makeNewOutputs(corr0, \
            {i: clstrs[i] for i in clstrs.keys() if i not in redoClusters}, clstrs2)
        # recalculate the evaluation metrics on this updated combined clustering
        newTstatsMean = np.mean([np.mean(silhNew[clstrsNew[i]])/ \
            np.std(silhNew[clstrsNew[i]]) for i in clstrsNew.keys()])
        # return the best clustering (either original or refined) 
        if newTstatsMean <= tStatMean:
            return corr1, clstrs, silh   # otherwise, keep the original clustering
        else:
            return corrNew, clstrsNew, silhNew # if the new combined clustering is better, return the new ones

experimental results